In [2]:
import pandas as pd
import numpy as np
import xarray as xr
import cdsapi 

# viz
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
from cartopy.io import shapereader
import plotly.graph_objects as go
import plotly.express as px
import folium
import seaborn as sns

import glob
import sys
import os

from scipy.interpolate import griddata
from numpy.polynomial.polynomial import Polynomial

In [3]:
import geopandas as gpd
import rioxarray
from shapely.geometry import mapping

def get_shape_us(shapefile, state_abbreviation):
    shapefile = shapefile.to_crs("EPSG:4326")
    state_shape = shapefile[shapefile.STUSPS == state_abbreviation].geometry
    return state_shape

def mask_xr_dataset(xr_data, shape):
    xr_data = xr_data.assign_coords(longitude=(((xr_data.longitude + 180) % 360) - 180)).sortby('longitude')
    xr_data.rio.write_crs("EPSG:4326", inplace=True)
    xr_data_masked = xr_data.rio.clip(shape.geometry.apply(mapping), shape.crs)
    return xr_data_masked

In [17]:
################################################
# MDK Final Severity Data preprocessing
################################################

ERA5 = xr.open_mfdataset('../../../data/UNSEEN/hurricane/ERA5_hourly/ERA5_????.nc', combine='by_coords')
lat_era5 = ERA5.latitude.values
lon_era5 = ERA5.longitude.values

LSMask = xr.open_dataset('../../../data/UNSEEN/hurricane/ERA5_land_sea_mask_new.nc')
LSMask['longitude'] = (((LSMask['longitude'] + 180) % 360) - 180)
LSMask_interp = LSMask.interp(latitude=lat_era5, longitude=lon_era5)

# uncomment below lines to filter for a specific state

# us_shapefile = gpd.read_file('../../../data/shapefiles/us/tl_2023_us_state.shp')
# shape = get_shape_us(us_shapefile, 'GA')
# filtered_era5 = mask_xr_dataset(ERA5, shape)

ERA5_land = (
    ERA5.where(LSMask_interp['lsm'].sel(time = '1981').squeeze('time') > 0.5)
)

# area_weights = np.cos(np.deg2rad(filtered_era5.latitude))
# area_weights = area_weights.expand_dims({'longitude': filtered_era5.longitude})

#filtered_era5['tp'] = filtered_era5['tp'] * area_weights

ERA5_coarsened = ERA5_land.coarsen(latitude=5, longitude=5, boundary='trim').sum()
ERA5_agg = ERA5_coarsened.resample(time='1D').sum()
ERA5_agg_monthly = ERA5_agg['tp'].resample(time='ME').max()
ERA5_agg_monthly_max = ERA5_agg_monthly.max(['latitude', 'longitude'])
ERA5_agg_monthly_max_df = ERA5_agg_monthly_max.to_dataframe().reset_index()
filtered_df = ERA5_agg_monthly_max_df.loc[ERA5_agg_monthly_max_df['time'].dt.month.between(6, 10)]
filtered_df['year'] = filtered_df['time'].dt.year
#filtered_df.to_csv('../../../data/UNSEEN/hurricane/rainfall_preprocessed/states/ERA5_1d_monthly_max.csv', index=False)

/var/folders/c_/5rx_wyrs0pg78x3j1_zxrpl00000gn/T/ipykernel_30266/3322754504.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['year'] = filtered_df['time'].dt.year
